In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import streamlit as st
import ast

session = get_active_session()

In [ ]:
create or replace database SUPPORT_DB;
create or replace schema SUPPORT_DATA;

In [ ]:
CREATE or REPLACE file format csvformat
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  type = 'CSV';

CREATE or REPLACE stage support_tickets_data_stage
  file_format = csvformat
  url = 's3://sfquickstarts/finetuning_llm_using_snowflake_cortex_ai/';

CREATE or REPLACE TABLE SUPPORT_TICKETS (
  ticket_id VARCHAR(60),
  customer_name VARCHAR(60),
  customer_email VARCHAR(60),
  service_type VARCHAR(60),
  request VARCHAR,
  contact_preference VARCHAR(60)
)
COMMENT = '{"origin":"sf_sit-is", "name":"aiml_notebooks_fine_tuning", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"sql"}}';

COPY into SUPPORT_TICKETS
  from @support_tickets_data_stage;

# Preview Support Tickets

In [ ]:
use database SUPPORT_DB;
use schema SUPPORT_DATA;

In [ ]:
df_support_tickets = session.table('SUPPORT_TICKETS')
df_support_tickets

## Custom email or text reponse generation for support tickets using LLMs

In [ ]:
prompt = """You are a customer support representative at a telecommunications company. 
Suddenly there is a spike in customer support tickets. 
You need to understand and analyze the support requests from customers.
Based on the root cause of the main issue in the support request, craft a response to resolve the customer issue.
Write a text message under 25 words, if the contact_preference field is text message.
Write an email in maximum of 100 words if the contact_preference field is email. 
Focus on alleviating the customer issue and improving customer satisfaction in your response.
Strictly follow the word count limit for the response. 
Write only email or text message response based on the contact_preference for every customer. 
Do not generate both email and text message response.
Always answer in Spanish.
"""


prompt1 = """
Please write an email or text promoting a new plan that will save customers total costs. 
Also resolve the customer issue based on the ticket category. 
If the contact_preference is text message, write text message response in less than 25 words. 
If the contact_preference is email, write email response in maximum 100 words.
Write only email or text message response based on the contact_preference for every customer.
Always answer in Spanish.
"""

In [ ]:
ticket_categories = ['Roaming fees', 'Slow data speed', 'Lost phone', 'Add new line', 'Closing account']

## Mistral-7b response

In [ ]:
df_mistral_7b_response_sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('mistral-7b',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as mistral_7b_response
from SUPPORT_TICKETS"""

df_mistral_7b_response = session.sql(df_mistral_7b_response_sql)
df_mistral_7b_response

## mistral-large response

In [ ]:
mistral_large_response_sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('mistral-large',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as mistral_large_response
from SUPPORT_TICKETS"""

df_mistral_large_response = session.sql(mistral_large_response_sql)
df_mistral_large_response

# Generate dataset to fine-tune mistral-7b

In [ ]:
# Stage 1: Filter by CONTACT_PREFERENCE
df_text = df_mistral_large_response.filter(F.col("CONTACT_PREFERENCE") == 'Text Message')
df_email = df_mistral_large_response.filter(F.col("CONTACT_PREFERENCE") == "Email")

# Stage 2: Apply word count logic
df_text_filtered = df_text.filter(F.regexp_count(F.col("MISTRAL_LARGE_RESPONSE"), r" ") <= 25)
df_email_filtered = df_email.filter(F.regexp_count(F.col("MISTRAL_LARGE_RESPONSE"), r" ") > 30)

# Combine the results
df_filtered = df_text_filtered.union(df_email_filtered)

df_filtered

In [ ]:
df_filtered.write.save_as_table("SUPPORT_TICKET_RESPONSES", mode="overwrite")  # "overwrite" can be changed to "append"

# Optional: Show the saved table to verify
saved_df = session.table("SUPPORT_TICKET_RESPONSES")
saved_df

In [ ]:
df_fine_tune = saved_df.with_column("prompt", 
                                    F.concat(F.lit(prompt), 
                                             F.lit(" "), 
                                             F.lit("Contact preference: "),
                                             F.col("contact_preference"), 
                                             F.lit(" "),
                                             F.col("ticket_category")))\
                        .select("ticket_id",
                                "prompt",
                                "mistral_large_response")

df_fine_tune.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style')

# Fine-tune mistral-7b

## Split data into training and evaluation

In [ ]:
train_df, eval_df = session.table("support_tickets_finetune_message_style").random_split(weights=[0.8, 0.2], seed=42)

train_df.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style_train')
eval_df.write.mode('overwrite').save_as_table('support_tickets_finetune_message_style_eval')

In [ ]:
session.table('support_tickets_finetune_message_style_train').show(1)

In [ ]:
session.table('support_tickets_finetune_message_style_eval').show(1)

## *Fine-tune mistral-7b model using Cortex*

In [ ]:
select snowflake.cortex.finetune('CREATE', 
'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B', 
'mistral-7b', 
'SELECT prompt, mistral_large_response as completion from support_tickets_finetune_message_style_train',
'SELECT prompt, mistral_large_response as completion from support_tickets_finetune_message_style_eval');

In [ ]:
select snowflake.cortex.finetune('SHOW');

### See status of the fine tuning job

In [ ]:
select snowflake.cortex.finetune('DESCRIBE', 'ft_afc1da8d-e14a-4c27-8af4-11bd6ccbacf2');

# Inference using fine-tuned model

In [ ]:
fine_tuned_model_name = 'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B'

sql = f""" select ticket_id, 
snowflake.cortex.classify_text(request, {ticket_categories}) as ticket_category,
contact_preference, 
trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}', request, ticket_category, contact_preference)),'\n') 
    as fine_tuned_mistral_7b_model_response
from SUPPORT_TICKETS"""

df_fine_tuned_mistral_7b_response = session.sql(sql)
df_fine_tuned_mistral_7b_response

## Streamlit application to auto-generate custom emails and text messages

In [ ]:
st.subheader("Generar automáticamente correos electrónicos o mensajes de texto personalizados")

with st.container():
    with st.expander("Ingrese la solicitud del cliente y seleccione LLM", expanded=True):
        #customer_request = st.text_area('Request',"""Viajé a Lima durante dos semanas y mantuve mi uso de datos al mínimo. Sin embargo, se me cobraron $90 en tarifas internacionales. Estos cargos no me fueron comunicados, y solicito un desglose detallado y un reembolso. Gracias.""")
        #Desde hace varios días, la velocidad de mi conexión a Internet ha sido extremadamente lenta, incluso cuando solo tengo un dispositivo conectado. No puedo ver videos en streaming ni hacer videollamadas sin interrupciones. He reiniciado el router varias veces, pero el problema persiste. Solicito una solución urgente o una revisión de mi servicio contratado.
        #ticket_categories = ['Roaming', 'Lentitud de Datos', 'Celular Perdido', 'Nueva Línea', 'Cerrar Cuenta']

        customer_request = st.text_area('Request', """Realicé un pago con mi tarjeta de crédito, pero la transacción fue rechazada sin razón aparente. Me gustaría entender el motivo y si hay algún bloqueo en mi cuenta. Agradecería una pronta solución.""")
        #Revisando los movimientos de mi cuenta, noté un cargo de 450.000 pesos que no reconozco y que nunca autoricé. Intenté comunicarme con el banco, pero no he recibido una respuesta clara sobre el proceso de disputa. Solicito una investigación urgente y el reembolso del monto debitado sin mi autorización.
        ticket_categories = ['Problema con Tarjeta', 'Fraude o Transacción No Reconocida', 'Solicitud de Crédito', 'Acceso a Banca Digital', 'Cierre de Cuenta']
        
        #customer_request = st.text_area('Solicitud', """Tuve una emergencia médica y acudí a un hospital dentro de la red de cobertura. Sin embargo, al momento del pago, me informaron que mi póliza no cubría ciertos procedimientos, a pesar de que en el contrato se indicaba lo contrario. Solicito una aclaración y el reembolso de los costos adicionales que tuve que asumir.""")
        #La semana pasada tuve una emergencia médica y llamé a la línea de asistencia de mi seguro para solicitar una ambulancia. Me dijeron que llegaría en 20 minutos, pero tardó más de una hora en llegar. Como consecuencia, tuve que buscar otro medio de transporte para llegar al hospital. Solicito una explicación sobre lo ocurrido y qué medidas tomarán para evitar estas demoras en el futuro.
        #ticket_categories = ['Reclamación de Seguro', 'Estado de Póliza', 'Cobertura de Beneficios', 'Cambio de Plan', 'Atención Médica y Asistencia']
         
        with st.container():
            left_col, right_col = st.columns(2)
            with left_col:
                selected_preference = st.selectbox('Seleccione preferencia de contacto', ('Text message', 'Email'), index=1)
            with right_col:
                selected_llm = st.selectbox('Seleccione LLM',('llama3-8b', 'mistral-7b', 'mistral-large', 'SUPPORT_MESSAGES_FINETUNED_MISTRAL_7B',), index=1)

with st.container():
    _,mid_col,_ = st.columns([.4,.3,.3])
    with mid_col:
        generate_template = st.button('Generar Mensaje ⚡',type="primary")

with st.container():
    if generate_template:
        category_sql = f"""
        select snowflake.cortex.classify_text('{customer_request}', {ticket_categories}) as ticket_category
        """
        df_category = session.sql(category_sql).to_pandas().iloc[0]['TICKET_CATEGORY']
        df_category_dict = ast.literal_eval(df_category)
        st.subheader("Categoría de Solicitud")
        st.write(df_category_dict['label'])

        message_sql = f"""
        select snowflake.cortex.translate(snowflake.cortex.complete('{selected_llm}',concat('{prompt}', '{customer_request}', '{selected_preference}')),'en','es') as custom_message
        """
        df_message = session.sql(message_sql).to_pandas().iloc[0]['CUSTOM_MESSAGE']
        st.subheader(selected_preference)
        st.write(df_message)